In [1]:
import pandas as pd

test = pd.read_csv('/home/lyuzhuoqi/projects/TLDR/data/paper_html_10.1038/abs_annotation/test.tsv', sep='\t')
deepseek_v3 = pd.read_csv('/home/lyuzhuoqi/projects/TLDR/data/paper_html_10.1038/abs_annotation/generated_annotations/deepseek_v3.txt', 
                          sep='\t', header=None, names=['deepseek_v3'])
gemma3 = pd.read_csv('/home/lyuzhuoqi/projects/TLDR/data/paper_html_10.1038/abs_annotation/generated_annotations/gemma3_TLDR.txt',
                          sep='\t', header=None, names=['gemma3'])
llama4 = pd.read_csv('/home/lyuzhuoqi/projects/TLDR/data/paper_html_10.1038/abs_annotation/generated_annotations/llama4_TLDR.txt',
                          sep='\t', header=None, names=['llama4'])
qwq = pd.read_csv('/home/lyuzhuoqi/projects/TLDR/data/paper_html_10.1038/abs_annotation/generated_annotations/qwq_TLDR.txt',
                          sep='\t', header=None, names=['qwq'])
df = test.join([deepseek_v3, gemma3, llama4, qwq])

for index in pd.read_csv("/home/lyuzhuoqi/projects/TLDR/description/invalid_entry_in_test.txt", sep='\t', header=None).values.flatten().tolist():
    df = df.drop(index-2)  # Adjusting for zero-based index
df

,abs doi,paper_id,abstract,annotation,deepseek_v3,gemma3,llama4,qwq
0,10.1073/pnas.91.7.2757,107202074,The origin and taxonomic status of domesticate...,A demonstration that cattle have been domestic...,This study provides genetic evidence for two i...,Reference 16 provides a molecular phylogeny of...,This study proposes that zebu and taurine catt...,This study uses mitochondrial DNA analysis to ...
1,10.1093/genetics/154.4.1785,83366887,Abstract The domestic pig originates from the ...,Evidence is presented for independent domestic...,This study provides genetic evidence for indep...,This study demonstrates independent domesticat...,This study reports the origin of European and ...,This study uses mitochondrial and nuclear DNA ...
2,10.1073/pnas.96.16.9252,122095374,We previously mapped a quantitative trait locu...,This paper shows how the identity-by-descent a...,This study reports the fine-mapping of a QTL f...,Reference 53 reports fine-mapping of a QTL for...,The QTL was fine-mapped to a 5 cM region on BT...,This study uses high-density marker genotyping...
3,10.1101/gr.10.2.220,100831446,A genome-wide linkage disequilibrium (LD) map ...,The pattern of linkage disequilibrium (LD) acr...,This study provides the first genome-wide LD m...,This study documents an unusually strong and l...,The authors provide evidence that linkage dise...,This study generates a genome-wide linkage dis...
4,10.1126/science.8134840,17452622,The European wild boar was crossed with the do...,The first paper to show the use of divergent i...,This study identifies a major QTL on chromosom...,This study identifies a major QTL on pig chrom...,This study reports a major QTL on SSC4 for fat...,This study identifies a major quantitative tra...
...,...,...,...,...,...,...,...,...
35631,10.2337/db08-1168,4860455,OBJECTIVE—Regulatory T-cells (Tregs) have cata...,This article describes the good manufacturing ...,This study demonstrates that CD4+CD127lo/−CD25...,This study provides a method for isolation and...,This study demonstrates that Tregs can be expa...,This study evaluates methods for isolating and...
35632,10.1126/science.aar3246,4860145,Engineering cytokine-receptor pairs Interleuki...,This study reports the generation of an orthog...,This study reports the engineering of orthogon...,This study describes an engineered IL-2/IL-2R ...,This study presents an engineered IL-2 partial...,This study engineers synthetic IL-2–receptor p...
35633,10.1126/science.aad2791,62290395,T cells target peptide combos One of the endur...,This article shows that some diabetogenic T ce...,This study identifies hybrid insulin peptides ...,This study reports the identification of hybri...,This study highlights the potential importance...,This study identifies hybrid peptides—covalent...
35634,10.1073/pnas.1902566116,82979762,Polymorphic HLAs form the primary immune barri...,This article describes the development of gene...,This study demonstrates a multiplex genome-edi...,This work reports on a cell engineering strate...,This study identifies a strategy for broad imm...,This study describes a dual strategy to engine...
